## config 설정 및 weight 초기화

In [1]:
import os
import torch
import random
import numpy as np
import matplotlib.pyplot
from bh_utils import *

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"





my_seed = 0
# weight_file_path = r"net_save\save_now_net_weights_20260112_121819_873.pth"
weight_file_path = None

# HYPER_DATASET = 'DVS_GESTURE'
HYPER_DATASET = 'N_MNIST'
# HYPER_DATASET = 'NTIDIGITS_TARGET_0'
# HYPER_DATASET = 'NTIDIGITS_TARGET_1'
# HYPER_DATASET = 'NTIDIGITS_TARGET_2'
# HYPER_DATASET = 'NTIDIGITS_TARGET_3'
# HYPER_DATASET = 'NTIDIGITS_TARGET_4'
# HYPER_DATASET = 'NTIDIGITS_TARGET_5'
# HYPER_DATASET = 'NTIDIGITS_TARGET_6'
# HYPER_DATASET = 'NTIDIGITS_TARGET_7'
# HYPER_DATASET = 'NTIDIGITS_TARGET_8'
# HYPER_DATASET = 'NTIDIGITS_TARGET_9'

if HYPER_DATASET == 'DVS_GESTURE':
    topN_at_gesture = 4
    assert topN_at_gesture in [0,4,12]
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?  # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 0},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 10},
        "input_size_layer1":              {"trigger": 6,  "value": 980}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 1},
        "binary_classifier_mode":         {"trigger": 8,  "value": 0},
        "loser_encourage_mode":           {"trigger": 9,  "value": 0}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 128.0, sg_width = 4.0 * 2, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 128.0, sg_width = 4.0 * 2, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'N_MNIST':
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 1},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 5},
        "input_size_layer1":              {"trigger": 6,  "value": 578}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 0},
        "loser_encourage_mode":           {"trigger": 9,  "value": 0}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 256.0, sg_width = 10.0 * 2, learning_rate = 2.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 256.0, sg_width = 10.0 * 2, learning_rate = 2.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_0':
    target_word_at_ntidigits = 0
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 256.0, sg_width = 3.75, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 256.0, sg_width = 3.75, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_1':
    target_word_at_ntidigits = 1
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 128.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 128.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_2':
    target_word_at_ntidigits = 2
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 2048.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 1024.0, sg_width = 8.0, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_3':
    target_word_at_ntidigits = 3
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 512.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 512.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_4':
    target_word_at_ntidigits = 4
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 1024.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 1024.0, sg_width = 1.0, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_5':
    target_word_at_ntidigits = 5
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 1024.0, sg_width = 4.0, learning_rate = 2.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 256.0, sg_width = 32.0, learning_rate = 2.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_6':
    target_word_at_ntidigits = 6
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 1024.0, sg_width = 2.5, learning_rate = 2.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 1024.0, sg_width = 2.5, learning_rate = 2.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_7':
    target_word_at_ntidigits = 7
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 32.0, sg_width = 2.0, learning_rate = 1.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 256.0, sg_width = 4.0, learning_rate = 1.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_8':
    target_word_at_ntidigits = 8
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 512.0, sg_width = 3.75, learning_rate = 2.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 512.0, sg_width = 3.75, learning_rate = 2.0, verbose = False)},
    }
elif HYPER_DATASET == 'NTIDIGITS_TARGET_9':
    target_word_at_ntidigits = 9
    p_config = { # Trigger는 다시 당겨지고..
        "asic_mode":                      {"trigger": 1,  "value": 0}, #안쓸듯?   # 0 training_only, 1 train_inf_sweep, 2 inference_only
        "training_epochs":                {"trigger": 2,  "value": 200}, #안쓸듯?
        "inference_epochs":               {"trigger": 3,  "value": 200}, #안쓸듯?
        "dataset":                        {"trigger": 4,  "value": 2},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
        "timesteps":                      {"trigger": 5,  "value": 8},
        "input_size_layer1":              {"trigger": 6,  "value": 512}, # DVS_GESTURE 980, N_MNIST 578, NTIDIGITS 512
        "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
        "binary_classifier_mode":         {"trigger": 8,  "value": 1},
        "loser_encourage_mode":           {"trigger": 9,  "value": 1}, # 1 true 0 false
        "layer1_cut_list":                {"trigger": 10, "value": layer_cut_generator(threshold = 512.0, sg_width = 3.75, learning_rate = 2.0, verbose = False)},
        "layer2_cut_list":                {"trigger": 11, "value": layer_cut_generator(threshold = 512.0, sg_width = 3.75, learning_rate = 2.0, verbose = False)},
    }
else:
    assert False, "HYPER_DATASET 설정 필요"



if p_config["dataset"]["value"] == 0:
    dataset = 'DVS_GESTURE' # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
elif p_config["dataset"]["value"] == 1:
    dataset = 'N_MNIST'
elif p_config["dataset"]["value"] == 2:
    dataset = 'NTIDIGITS'
else:
    assert False, "데이터셋 설정 필요"

# 32bit per address
dram_address_weight_start = 0 
dram_address_weight_last = 110_392



seed_assign(my_seed)
if dataset == 'DVS_GESTURE':
    input_size = p_config['input_size_layer1']['value']
    assert input_size == 980, "DVS_GESTURE input_size는 980이어야 합니다."
    init_scaling = [10000+ 9,10000+ 9,10000+ 8]
elif dataset == 'N_MNIST':
    input_size = p_config['input_size_layer1']['value']
    assert input_size == 578, "N_MNIST input_size는 578이어야 합니다."
    init_scaling = [10000+ 9,10000+ 9,10000+ 8]
elif dataset == 'NTIDIGITS':
    input_size = p_config['input_size_layer1']['value']
    assert input_size == 512, "NTIDIGITS input_size는 512이어야 합니다."
    init_scaling = [10000+ 12,10000+ 12,10000+ 11]
else:
    assert False, "데이터셋 설정 필요"  


layer1_sram_row = 980
layer1_sram_bank = 10
layer1_sram_bit = 160
layer2_sram_row = 200
layer2_sram_bank = 10
layer2_sram_bit = 160
layer3_sram_row = 200
layer3_sram_bank = 10
layer3_sram_bit = 8



if (weight_file_path is not None) :
    if not os.path.exists(weight_file_path):
        print("파일을 찾을 수 없습니다.")
    else:
        # 파일 로드
        # weights = torch.load(weight_file_path, map_location=torch.device('cpu'))
        weights = torch.load(weight_file_path, map_location=torch.device('cpu'), weights_only=True)
        # print(f"=== 'fc.weight' 필터링 결과 ===\n")
        # print(f"{'Layer Name':<40} | {'Shape'}")
        # print("-" * 60)

        if isinstance(weights, dict):
            for key, value in weights.items():
                if "module.layers.1.fc.weight" in key:
                    # weight_plot(value)
                    if dataset == 'DVS_GESTURE':
                        assert value.shape == (200,980), f"value.shape: {value.shape}"
                    elif dataset == 'N_MNIST':
                        assert value.shape == (200,578), f"value.shape: {value.shape}"
                    elif dataset == 'NTIDIGITS':
                        assert value.shape == (200,512), f"value.shape: {value.shape}"
                    else:
                        assert False, "데이터셋 설정 필요"  

                    layer1_weight = torch.nn.functional.pad(value, (0, layer1_sram_row - value.shape[1], 0, 0), "constant", 0)
                    # print(f"{key:<40} | {list(layer1_weight.shape)}")
                    # print(layer1_weight)
                if "module.layers.4.fc.weight" in key:
                    assert value.shape == (200,200), f"value.shape: {value.shape}"
                    # weight_plot(value)
                    layer2_weight = value
                    # print(f"{key:<40} | {list(layer2_weight.shape)}")
                    # print(layer2_weight)
                if "module.layers.7.fc.weight" in key:
                    assert value.shape == (10,200), f"value.shape: {value.shape}"
                    # weight_plot(value)
                    layer3_weight = value
                    # print(f"{key:<40} | {list(layer3_weight.shape)}")
                    # print(layer3_weight)
        else:
            print("딕셔너리 형태가 아닙니다.")


else:
    layer1_value = torch.nn.Linear(input_size, layer2_sram_row, bias=False)
    layer1_weight = layer1_value.weight.data * (2**(init_scaling[0]-10000) * 1.0) if init_scaling[0] >= 10000 else layer1_value.weight.data * (2**(init_scaling[0]) * 1.0)
    layer1_weight = quantize_tensor(layer1_weight)
    # weight_plot(layer1_weight)
    layer1_weight = torch.nn.functional.pad(layer1_weight, (0, layer1_sram_row - layer1_weight.shape[1], 0, 0), "constant", 0)
    # print(layer1_weight, layer1_weight.shape)


    layer2_value = torch.nn.Linear(layer2_sram_row, layer3_sram_row, bias=False)
    layer2_weight = layer2_value.weight.data * (2**(init_scaling[1]-10000) * 1.0) if init_scaling[0] >= 10000 else layer1_value.weight.data * (2**(init_scaling[0]) * 1.0)
    layer2_weight = quantize_tensor(layer2_weight)
    # weight_plot(layer2_weight)
    # print(layer2_weight, layer2_weight.shape)


    layer3_value = torch.nn.Linear(layer3_sram_row, 10, bias=False)
    layer3_weight = layer3_value.weight.data * (2**(init_scaling[2]-10000) * 1.0) if init_scaling[0] >= 10000 else layer1_value.weight.data * (2**(init_scaling[0]) * 1.0)
    layer3_weight = quantize_tensor(layer3_weight)
    # weight_plot(layer3_weight)
    # print(layer3_weight, layer3_weight.shape)

layer1_weight_np = layer1_weight.numpy().astype(np.int8)
layer2_weight_np = layer2_weight.numpy().astype(np.int8)
layer3_weight_np = layer3_weight.numpy().astype(np.int8)


layer1_weight_np_t = layer1_weight_np.T

# 세로(axis=1) 방향으로 10등분
# 200열 / 10개 뱅크 = 각 뱅크당 20열
layer1_banks = np.split(layer1_weight_np_t, layer1_sram_bank, axis=1)
layer1_banks_renewal = np.split(layer1_weight_np_t.copy(), layer1_sram_bank, axis=1)
for i in range(layer1_banks[0].shape[0]):
    for j in range(len(layer1_banks)):
        idx1 = (j + (i%10))%10
        (layer1_banks_renewal[idx1])[i] = (layer1_banks[j])[i]

# axis=1(열 방향)로 10개의 뱅크를 다시 이어붙임
layer1_weight_reassembled = np.concatenate(layer1_banks_renewal, axis=1) # (980, 200)



layer2_weight_np_t = layer2_weight_np.T

layer2_banks = np.split(layer2_weight_np_t, layer2_sram_bank, axis=1)
layer2_banks_renewal = np.split(layer2_weight_np_t.copy(), layer2_sram_bank, axis=1)
for i in range(layer2_banks[0].shape[0]):
    for j in range(len(layer2_banks)):
        idx1 = (j + (i%10))%10
        (layer2_banks_renewal[idx1])[i] = (layer2_banks[j])[i]

layer2_weight_reassembled = np.concatenate(layer2_banks_renewal, axis=1) 


layer3_weight_np_t = layer3_weight_np.T

layer3_banks = np.split(layer3_weight_np_t, layer3_sram_bank, axis=1)
layer3_banks_renewal = np.split(layer3_weight_np_t.copy(), layer3_sram_bank, axis=1)
for i in range(layer3_banks[0].shape[0]):
    for j in range(len(layer3_banks)):
        idx1 = (j + (i%10))%10
        (layer3_banks_renewal[idx1])[i] = (layer3_banks[j])[i]

layer3_weight_reassembled = np.concatenate(layer3_banks_renewal, axis=1) 

# print(f'layer1_weight_reassembled shape: {layer1_weight_reassembled.shape}')
# print(f'layer2_weight_reassembled shape: {layer2_weight_reassembled.shape}')
# print(f'layer3_weight_reassembled shape: {layer3_weight_reassembled.shape}')


# 결과를 저장할 리스트 (또는 파일로 직접 저장 가능)
layer1_hex_outputs = []
for row in range(layer1_weight_reassembled.shape[0]):
    for bank in range(layer1_sram_bank):
        # 1. 현재 뱅크의 20바이트 추출
        one_dram_word = layer1_weight_reassembled[row, bank*20 : (bank+1)*20]
        
        # 2. 역순 정렬: [19][18]...[0] 순서로 hex 문자열 생성
        # 각 요소는 8bit이므로 2자리 hex(:02x)로 변환
        hex_160bit = "".join(f"{int(val) & 0xFF:02x}" for val in reversed(one_dram_word))
        
        # 3. 256bit가 되도록 상위에 96bit(hex 24자리) 0 패딩 추가
        # 256bit = 64 hex chars (160bit = 40 hex chars)
        padding_96bit = "0" * 24
        final_256bit_word = padding_96bit + hex_160bit
        
        layer1_hex_outputs.append(final_256bit_word)
    
# 결과를 저장할 리스트 (또는 파일로 직접 저장 가능)
layer2_hex_outputs = []
for row in range(layer2_weight_reassembled.shape[0]):
    for bank in range(layer2_sram_bank):
        # 1. 현재 뱅크의 20바이트 추출
        one_dram_word = layer2_weight_reassembled[row, bank*20 : (bank+1)*20]
        
        # 2. 역순 정렬: [19][18]...[0] 순서로 hex 문자열 생성
        # 각 요소는 8bit이므로 2자리 hex(:02x)로 변환
        hex_160bit = "".join(f"{int(val) & 0xFF:02x}" for val in reversed(one_dram_word))
        
        # 3. 256bit가 되도록 상위에 96bit(hex 24자리) 0 패딩 추가
        # 256bit = 64 hex chars (160bit = 40 hex chars)
        padding_96bit = "0" * 24
        final_256bit_word = padding_96bit + hex_160bit
        
        layer2_hex_outputs.append(final_256bit_word)
    
# 결과를 저장할 리스트 (또는 파일로 직접 저장 가능)
layer3_hex_outputs = []
for row in range(layer3_weight_reassembled.shape[0]):
    for bank in range(layer3_sram_bank):
        # 1. 현재 뱅크의 1바이트 추출
        one_dram_word = layer3_weight_reassembled[row, bank*1 : (bank+1)*1]
        
        # 2. 역순 정렬: [19][18]...[0] 순서로 hex 문자열 생성
        # 각 요소는 8bit이므로 2자리 hex(:02x)로 변환
        hex_8bit = "".join(f"{int(val) & 0xFF:02x}" for val in reversed(one_dram_word))
        
        # 3. 256bit가 되도록 상위에 248bit(hex 62자리) 0 패딩 추가
        # 256bit = 64 hex chars (248bit = 62 hex chars)
        padding_248bit = "0" * 62
        final_256bit_word = padding_248bit + hex_8bit
        
        layer3_hex_outputs.append(final_256bit_word)
# print(f'layer1_hex_outputs length: {len(layer1_hex_outputs)}')
# print(f'layer2_hex_outputs length: {len(layer2_hex_outputs)}')
# print(f'layer3_hex_outputs length: {len(layer3_hex_outputs)}')

total_hex_outputs = layer1_hex_outputs + layer2_hex_outputs + layer3_hex_outputs
# print(f'total_hex_outputs length: {len(total_hex_outputs)}')


# total_hex_outputs의 요소들을 역순으로 뒤집어서(reversed) 하나로 합칩니다.
# 결과물은 엄청나게 긴 하나의 Hex 문자열이 됩니다.
# total_long_hex_string = "".join(reversed(total_hex_outputs))
total_long_hex_string = "".join(total_hex_outputs)

# # 출력 확인 (너무 길 수 있으니 앞뒤 일부와 전체 길이만 확인)
# print(f"Total String Length (Hex chars): {len(total_long_hex_string)}")
# print(f"Total String Length (Bits): {len(total_long_hex_string) * 4}bit")
# print(f"Total String Length (Bytes): {len(total_long_hex_string) * 4 / 8:.3f}Byte")
# print(f"Total String Length (kBytes): {len(total_long_hex_string) * 4 / 8/ 1024:.3f}kByte")
# print(f"Total String Length (MBytes): {len(total_long_hex_string) * 4 / 8/ 1024/1024:.3f}MByte")
# print(f'DRAM Address Range: 0 to {len(total_long_hex_string) * 4 / 256 *8 - 8:.0f}')
print(f"\nWeight 약간보자")
print(f"Start (MSB side): {total_long_hex_string[:256]}...")
print(f"End (LSB side): ...{total_long_hex_string[-256:]}")

# total_long_hex_string을 하나의 긴 문자열로 그대로 저장합니다.
with open("init_weight.txt", "w") as f:
    f.write(total_long_hex_string)

print(f"✅ 파일 저장 완료: init_weight.txt")
print(f"총 Hex 글자 수: {len(total_long_hex_string):,}자")
print(f"파일 용량: {len(total_long_hex_string) / 2:,.0f}B")
print(f"파일 용량: {len(total_long_hex_string) / 2 / 1024 / 1024:.3f}MB")

# # # 1. 파일 다시 읽기
# with open("init_weight.txt", "r") as f:
#     loaded_hex_string = f.read()

# # 2. 원본 데이터와 비교
# is_identical = (total_long_hex_string == loaded_hex_string)

# # 3. 결과 출력
# print(f"=== 검증 결과 ===")
# print(f"원본 문자열 길이: {len(total_long_hex_string):,} 자")
# print(f"읽어온 문자열 길이: {len(loaded_hex_string):,} 자")
# print(f"데이터 일치 여부: {is_identical}")

# # 만약 다르다면 어느 부분이 다른지 체크 (데이터가 같으면 실행 안 됨)
# if not is_identical:
#     for i in range(min(len(total_long_hex_string), len(loaded_hex_string))):
#         if total_long_hex_string[i] != loaded_hex_string[i]:
#             print(f"최초로 다른 지점: 인덱스 {i}")
#             print(f"원본: {total_long_hex_string[i:i+10]}...")
#             print(f"파일: {loaded_hex_string[i:i+10]}...")
#             break


Weight 약간보자
Start (MSB side): 000000000000000000000000f8fc0209ed14effafbff15020efc05ec0cf70900000000000000000000000000feecf5f5f5f1ed0ef512121408f2f1f4f6fa11f500000000000000000000000003efebff09fb00f71102f511f2fa14fdebec11f60000000000000000000000000bf7f201f9f50bf504fd09f1f7f6fe0cf8f8060c...
End (LSB side): ...0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000f800000000000000000000000000000000000000000000000000000000000000f7
✅ 파일 저장 완료: init_weight.txt
총 Hex 글자 수: 883,200자
파일 용량: 441,600B
파일 용량: 0.421MB


## config 및 weight 값 fpga 삽입

In [2]:
import time
import numpy as np
from mms_ok import XEM7310
from mms_ok import XEM7360
from secrets import token_hex
import torch
import random
from bh_utils import *


bitstream_path = r"bitstream/top_bh_fpga.bit"
fpga = XEM7360(bitstream_path=bitstream_path)
# fpga.close() 
def bh_btpipein(data, block_size_candidate = [16384, 8192, 4096, 2048, 1024, 512, 256, 128, 64, 32, 16], verbose = True):
# def bh_btpipein(data, block_size_candidate = [16384, 16], verbose = False):
    total_start = time.time()

    left_string_len = len(data)
    read_string_len = 0

    if (verbose):
        print(f"\n\n")
        print(f"bh_btpipein VERBOSE")

    for bl_size in block_size_candidate:
        if (left_string_len >= bl_size*2):
            read_string_len_temp = (left_string_len // (bl_size*2)) * (bl_size*2)
            sent_bytes = read_string_len_temp // 2
            
            # --- 각 블록 전송 시간 및 속도 측정 ---
            step_start = time.time()
            # data_temp = data[read_string_len : read_string_len + read_string_len_temp]
            fpga.WriteToBlockPipeIn(0x80, data[read_string_len : read_string_len + read_string_len_temp], block_size=bl_size, reorder_str=True)

            step_end = time.time()
            duration = step_end - step_start
            
            # MiB/s 계산 (bytes / 1024^2 / sec)
            speed = (sent_bytes / (1024 * 1024)) / duration if duration > 0 else 0

            if (verbose):
                print(f"{bl_size:10d} byte | {duration*1000:10.4f} ms | {speed:12.2f} MiB/s | {sent_bytes:12d} bytes")
                print(f'data_temp 뒷 네글자: {(data[read_string_len : read_string_len + read_string_len_temp])[-4:]}')

            left_string_len -= read_string_len_temp
            read_string_len += read_string_len_temp

    total_end = time.time()
    assert left_string_len == 0 and read_string_len == len(data), f"left_string_len: {left_string_len}, read_string_len: {read_string_len}, len(data): {len(data)}"
    if (verbose):
        print("-" * 30)
        print(f"Total Time: {total_end - total_start:.4f} sec")
        print(f"Total Bytes: {read_string_len//2} bytes")
        print(f"bh_btpipein VERBOSE END\n\n")
# ######### Reset ######################################################################
# ######### Reset ######################################################################
# ######### Reset ######################################################################
# ######### Reset ######################################################################
reset_time = 0.5  # seconds
print(f'Resetting FPGA for {reset_time} seconds...')
fpga.reset(reset_address = 0x00, reset_time = reset_time, active_low = True)
print('System reset complete.')
time.sleep(0.5) # 이거 꼭 해라
fpga.CheckTriggered(0x60, 1, timeout=10.0) # config 다됐냐
# ######### Reset ######################################################################
# ######### Reset ######################################################################
# ######### Reset ######################################################################
# ######### Reset ######################################################################

print('')
print('')
print('')
print('')

# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')

# ######### p_config mode ######################################################################
p_config_ongoing_sleep_time = 0.01  # seconds
p_config_verbose = False
# config mode on
print(f'######### P_STATE_01_WORKLOAD_CONFIG mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# asic_mode: 0 training_only, 1 train_inf_sweep, 2 inference_only 
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['asic_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['asic_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['asic_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- asic_mode')
time.sleep(p_config_ongoing_sleep_time)
# training_epochs
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['training_epochs']['value']))
fpga.ActivateTriggerIn(0x40, p_config['training_epochs']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['training_epochs']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- training_epochs')
time.sleep(p_config_ongoing_sleep_time)
# inference_epochs
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['inference_epochs']['value']))
fpga.ActivateTriggerIn(0x40, p_config['inference_epochs']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['inference_epochs']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- inference_epochs')
time.sleep(p_config_ongoing_sleep_time)
# dataset: 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['dataset']['value']))
fpga.ActivateTriggerIn(0x40, p_config['dataset']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['dataset']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- dataset')
time.sleep(p_config_ongoing_sleep_time)
# timesteps
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['timesteps']['value']))
fpga.ActivateTriggerIn(0x40, p_config['timesteps']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['timesteps']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- timesteps')
time.sleep(p_config_ongoing_sleep_time)
# input_size_layer1_define
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['input_size_layer1']['value']))
fpga.ActivateTriggerIn(0x40, p_config['input_size_layer1']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['input_size_layer1']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- input_size_layer1')
time.sleep(p_config_ongoing_sleep_time)
# long_time_input_streaming_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['long_time_input_streaming_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['long_time_input_streaming_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['long_time_input_streaming_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- long_time_input_streaming_mode')
time.sleep(p_config_ongoing_sleep_time)
# binary_classifier_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['binary_classifier_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['binary_classifier_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['binary_classifier_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- binary_classifier_mode')
time.sleep(p_config_ongoing_sleep_time)
# loser_encourage_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['loser_encourage_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['loser_encourage_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['loser_encourage_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
if p_config_verbose:
    print(f'WireOut 0x20 value: {ep20wireout} <-- loser_encourage_mode')
time.sleep(p_config_ongoing_sleep_time)
# layer1_cut_list
for i in range(15):
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer1_cut_list']['value'][i]))
    fpga.ActivateTriggerIn(0x40, p_config['layer1_cut_list']['trigger'])
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer1_cut_list']['trigger']))
    time.sleep(p_config_ongoing_sleep_time)
    ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
    if p_config_verbose:
        print(f'WireOut 0x20 value: {ep20wireout} <-- layer1_cut_list[{i}]')
# layer2_cut_list
for i in range(15):
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['value'][i]))
    fpga.ActivateTriggerIn(0x40, p_config['layer2_cut_list']['trigger'])
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['trigger']))
    time.sleep(p_config_ongoing_sleep_time)
    ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
    if p_config_verbose:
        print(f'WireOut 0x20 value: {ep20wireout} <-- layer2_cut_list[{i}]')
# ######### p_config mode######################################################################
        
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))

fpga.ActivateTriggerIn(0x40, 31)

fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐




    
print('')
print('')
print('')
print('')

# ######### p_config done mode ######################################################################
print(f'######### P_STATE_02_WORKLOAD_CONFIG_DONE mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(2))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ######### p_config done mode ######################################################################


    
print('')
print('')
print('')
print('')



# ######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################
print(f'######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(3))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


print(f'\nDRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!')
print(f'dram_address_weight_start: {dram_address_weight_start:,}')
print(f'dram_address_weight_last: {dram_address_weight_last:,}')












# #DRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_weight_start)) # dram write address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_weight_last)) # dram write address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐
# data = "01"*4 + "02"*4 + "03"*4 + "04"*4 + "05"*4 + "06"*4 + "07"*4 + "08"*4 + "09"*4 + "0A"*4 + "0B"*4 + "0C"*4 + "0D"*4 + "0E"*4 + "0F"*4 + "10"*4
# data = "01"*4 + "02"*4 + "03"*4 + "04"*4 + "05"*4 + "06"*4 + "07"*4 + "08"*4 + "09"*4 + "0A"*4 + "0B"*4 + "0C"*4 + "0D"*4 + "0E"*4 + "0F"*4 + "0F"*4
with open("init_weight.txt", "r") as f:
    loaded_hex_string = f.read()
data = loaded_hex_string
# data = loaded_hex_string[-64:]




# #DRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!
# check_n_bits = 256 *      3  
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0)) # dram write address
# fpga.ActivateTriggerIn(0x40, 30)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0 + (check_n_bits-256)//32 )) # dram write address last
# fpga.ActivateTriggerIn(0x40, 30)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐
# with open("init_weight.txt", "r") as f:
#     loaded_hex_string = f.read()
# # data = "01"*4 + "02"*4 + "03"*4 + "04"*4 + "05"*4 + "06"*4 + "07"*4 + "08"*4 + "09"*4 + "0A"*4 + "0B"*4 + "0C"*4 + "0D"*4 + "0E"*4 + "0F"*4 + "FF"*4
# # data = "09"*4 + "0A"*4 + "0B"*4 + "0C"*4 + "0D"*4 + "0E"*4 + "0F"*4 + "FF"*4
# # data = "09"*4 + "0A"*4 + "0B"*4 + "0C"*4 + "0D"*4 + "0E"*4 + "0F"*4 + "0B"*4
# data = token_hex(check_n_bits//8) # byte
# print(f'테스트 data[-2:0] {data[-2:0]}', f'data length: {len(data)}')
# print(f'테스트 data[-2:0] {data[-2:0]}', f'data length: {len(data)}')
# print(f'테스트 data[-2:0] {data[-2:0]}', f'data length: {len(data)}')







# print(f'datain string length: {len(data)}') # 883200 (883200*2 byte)

# # block size (byte)  (len(data)*2) 가 아래숫자에 배수여야함.
# # USB 2.0 FullSpeed - Multiple of two [2..64]
# # USB 2.0 HighSpeed - Multiple of two [2..1024]
# # USB 3.0 FullSpeed - Power of two [16..64]
# # USB 3.0 HighSpeed - Power of two [16..1024]
# # USB 3.0 SuperSpeed - Power of two [16..16384]
# block_size_candidate = [16384, 8192, 4096, 2048, 1024, 512, 256, 128, 64, 32, 16]
# block_size_candidate = [16384, 16]
# block_size_candidate = [16384, 32]
# block_size_candidate = [16384, 64]
# block_size_candidate = [16]
# verbose = True
# bh_btpipein(data, block_size_candidate = [16384, 64], verbose = True) # block_size_candidate should be >= 64 and 2의 승수.



############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
bh_btpipein(data, verbose = True) # block_size_candidate should be >= 64 and 2의 승수.
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################



fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐
# ######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################







print('')
print('')
print('')
print('')









print(f'######### P_STATE_04_DRAMFILL_WEIGHT_DATA_DONE mode ######################################################################')
# ######### P_STATE_04_DRAMFILL_WEIGHT_DATA_DONE mode ######################################################################

fpga.SetWireInValue(0x01, signed_encoding_32bit(12))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'wr_en cnt value = WireOut 0x20 value: {ep20wireout}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))

# fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ######### P_STATE_04_DRAMFILL_WEIGHT_DATA_DONE mode ######################################################################







print('')
print('')
print('')
print('')















2026-01-27 15:23:22.496 | INFO     | mms_ok.ok_setup:copy_frontpanel_files:28 - FrontPanel SDK Version: 5.3.6
2026-01-27 15:23:22.501 | INFO     | mms_ok.ok_setup:copy_frontpanel_files:39 - FrontPanel API ready
2026-01-27 15:23:22.757 | INFO     | mms_ok.fpga:_validate_bitstream_path:117 - Bitstream file: c:\Users\bhkim\measurement_setting\bitstream\top_bh_fpga.bit
2026-01-27 15:23:22.758 | INFO     | mms_ok.fpga:_validate_bitstream_path:120 - Bitstream date: 2026-01-27 14:20:51
2026-01-27 15:23:22.765 | INFO     | mms_ok.fpga:_connect:147 - Model        : XEM7360-K160T
2026-01-27 15:23:22.766 | INFO     | mms_ok.fpga:_connect:148 - Serial Number: 2420001BL5
2026-01-27 15:23:22.766 | INFO     | mms_ok.fpga:_connect:149 - Interface    : USB 3
2026-01-27 15:23:22.767 | INFO     | mms_ok.fpga:_connect:150 - USB Speed    : SUPER
2026-01-27 15:23:22.768 | INFO     | mms_ok.fpga:_connect:151 - Max Blocksize: 16384
2026-01-27 15:23:22.768 | INFO     | mms_ok.fpga:_connect:152 - Wire Width   :

Resetting FPGA for 0.5 seconds...


2026-01-27 15:23:23.728 | INFO     | mms_ok.fpga:reset:272 - Reset End (0.5s)



System reset complete.




######### P_STATE_01_WORKLOAD_CONFIG mode ######################################################################




######### P_STATE_02_WORKLOAD_CONFIG_DONE mode ######################################################################




######### P_STATE_03_DRAMFILL_WEIGHT_DATA mode ######################################################################

DRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!
dram_address_weight_start: 0
dram_address_weight_last: 110,392



bh_btpipein VERBOSE
     16384 byte |    72.7067 ms |         5.59 MiB/s |       425984 bytes
data_temp 뒷 네글자: 000e
      8192 byte |     2.0008 ms |         3.90 MiB/s |         8192 bytes
data_temp 뒷 네글자: 000d
      4096 byte |     0.0000 ms |         0.00 MiB/s |         4096 bytes
data_temp 뒷 네글자: 000a
      2048 byte |     0.0000 ms |         0.00 MiB/s |         2048 bytes
data_temp 뒷 네글자: 00ff
      1024 byte |     0.0000 ms |         0.00 MiB/s |         1024 bytes
data_temp 뒷 네글자: 000e
       256 

In [3]:
print('dram_address_weight_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_weight_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')


print('write count before trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
fpga.ActivateTriggerIn(0x40, 29)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
print('write count after trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('fifo_p2d_data_wr_cnt')
fpga.SetWireInValue(0x01, signed_encoding_32bit(12))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('쓴 데이터의 마지막 16bit')
fpga.SetWireInValue(0x01, signed_encoding_32bit(14))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# 수정 전: unsigned hex: 0xf659
# 수정 후: unsigned hex: f659 (딱 4자리)
string_from_fpga = f'{fpga.GetWireOutValue(0x20):04x}'
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {string_from_fpga}')
print('넣은거  ', data[-4:])
print('넣은 거랑 똑같음?',str(data[-4:]) == str(string_from_fpga))
# print(f'len(data): {len(data)}')
if str(data[-4:]) != str(string_from_fpga):
    if  str(string_from_fpga) in str(data):
        #몇번째에
        idx = data.index(str(string_from_fpga))
        print(f'안에있음, {idx}번째에 있음')
        print(f'len(data) - idx: {len(data) - idx}')
    else:
        print('안에없음')
    

fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


dram_address_weight_start
value = WireOut 0x20 value: 0 unsigned hex: 0x0
dram_address_weight_last
value = WireOut 0x20 value: 110392 unsigned hex: 0x1af38
write count before trigging (하위 17비트)
value = WireOut 0x20 value: 0 unsigned hex: 0x0
write count after trigging (하위 17비트)
value = WireOut 0x20 value: 110400 unsigned hex: 0x1af40
fifo_p2d_data_wr_cnt
value = WireOut 0x20 value: 110400 unsigned hex: 0x1af40
쓴 데이터의 마지막 16bit
value = WireOut 0x20 value: 247 unsigned hex: 00f7
넣은거   00f7
넣은 거랑 똑같음? True


0

## inference data 삽입

In [4]:




print(f'######### P_STATE_05_DRAMFILL_INFERENCE_DATA mode ######################################################################')
# ######### P_STATE_05_DRAMFILL_INFERENCE_DATA mode ######################################################################
fpga.SetWireInValue(0x01, signed_encoding_32bit(5))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))




if dataset == 'DVS_GESTURE':
    bit_per_sample = 9984
    inf_samples = 240
    tr_samples = 979
    tr_epochs_in_dram = 200

    # 9984 bit * 240 samples
    dram_address_inference_start = 110_400
    dram_address_inference_last =  dram_address_inference_start + ((bit_per_sample * inf_samples) // 32) - 8 # 185_272

    # 9984 bit * 979 samples * 200 epochs
    dram_address_training_start = 1_070_400
    dram_address_training_last = dram_address_training_start + (((bit_per_sample * tr_samples) // 32)*tr_epochs_in_dram) - 8  # 62_159_992

    inference_data_path = f"zz_measure_dataset/gesture/topN_{topN_at_gesture}/seed_{my_seed}/test"
    training_data_path = f"zz_measure_dataset/gesture/topN_{topN_at_gesture}/seed_{my_seed}/train"
    file_name_prefix = "gesture"

elif dataset == 'N_MNIST':
    # training 때 25epochs만넣을거임 nmnist는!!!!!!!!!!!!!!!!!!!!!!!!!!!

    bit_per_sample = 3072
    inf_samples = 10000
    tr_samples = 60000
    tr_epochs_in_dram = 25

    # 3072 bit * 10000 samples
    dram_address_inference_start = 110_400
    dram_address_inference_last = dram_address_inference_start + ((bit_per_sample * inf_samples) // 32) - 8 # 1_070_392

    # 3072 bit * 60000 samples * 25 epochs
    dram_address_training_start = 1_070_400
    dram_address_training_last = dram_address_training_start + (((bit_per_sample * tr_samples) // 32)*tr_epochs_in_dram) - 8 # 145_070_392

    inference_data_path = f"zz_measure_dataset/nmnist/test"
    training_data_path = f"zz_measure_dataset/nmnist/train"
    file_name_prefix = "nmnist"

elif dataset == 'NTIDIGITS':

    bit_per_sample = 4352
    inf_samples = 452
    if target_word_at_ntidigits in [0]:
        tr_samples = 4032
    elif target_word_at_ntidigits in [1,2,3,4,5,6,7,8,9]:
        tr_samples = 4030
    else:
        assert False, "NTIDIGITS target word 설정 필요"
    tr_epochs_in_dram = 200

    # 4352 bit * n samples
    dram_address_inference_start = 110_400
    dram_address_inference_last = dram_address_inference_start + ((bit_per_sample * inf_samples) // 32) - 8 # 171,864

    # 4352 bit * n samples * 200 epochs
    dram_address_training_start = 1_070_400
    dram_address_training_last = dram_address_training_start + (((bit_per_sample * tr_samples) // 32)*tr_epochs_in_dram) - 8  #110,740,792 #at word0 110,686,392

    inference_data_path = f"zz_measure_dataset/ntidigits/word{target_word_at_ntidigits}/test"
    training_data_path = f"zz_measure_dataset/ntidigits/word{target_word_at_ntidigits}/train"
    file_name_prefix = f"ntidigits_word{target_word_at_ntidigits}"

else:
    assert False, "데이터셋 설정 필요"

saved_path = f"{inference_data_path}/01_total_inference_data.txt"
if os.path.exists(saved_path) == True:
    print('이미 total inference dataset 파일이 존재하여 그것을 불러옵니다.', f'path : {saved_path}')
    with open(saved_path, "r") as f:
        total_inference_dataset_hex_string = f.read()
else:
    print('total inference dataset 파일이 존재하지 않아 새로 만듭니다.', f'path : {saved_path}')
    # inference making
    inference_dataset = []
    for sample_idx in range(inf_samples):
        sample_path = f"{inference_data_path}/{file_name_prefix}_{sample_idx}.txt"
        
        # 파일이 존재하는지 확인
        if not os.path.exists(sample_path):
            print(f"파일을 찾을 수 없습니다: {sample_path}. 루프를 종료합니다.")
            break
            
        with open(sample_path, "r") as f:
            sample_hex_string = f.read()
            inference_dataset.append(sample_hex_string)
    total_inference_dataset_hex_string = "".join(inference_dataset)
    with open(saved_path, "w") as f:
        f.write(total_inference_dataset_hex_string)

with open("total_inference_dataset_hex_string.txt", "w") as f:
    f.write(total_inference_dataset_hex_string)

with open("total_inference_dataset_hex_string.txt", "r") as f:
    total_inference_dataset_hex_string = f.read()

print(f'len(total_inference_dataset_hex_string): {len(total_inference_dataset_hex_string)} x 4bit')
print(f'dram_address_inference_last - dram_address_inference_start + 8: {dram_address_inference_last - dram_address_inference_start + 8} x 32bit')






print(f'\nDRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!')
print(f'dram_address_inference_start: {dram_address_inference_start:,}')
print(f'dram_address_inference_last: {dram_address_inference_last:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_inference_start)) # dram write address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_inference_last)) # dram write address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐




print('\nwrite count before trigging', '(하위 17비트)' , 'address 재설정하면 0으로 리셋 돼 있음. trigging해보셈.')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
fpga.ActivateTriggerIn(0x40, 29)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
print('write count after trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')







print('\n\ndram address 잘 드갔나?')
print('dram_address_weight_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_weight_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
print('\n')
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))














print('dataset의 앞 64글자와 뒤 64글자')
print(f"Start (MSB side): {total_inference_dataset_hex_string[:64]}...")
print(f"End (LSB side): ...{total_inference_dataset_hex_string[-64:]}")



data = total_inference_dataset_hex_string





print(f'data length: {len(data)}')

############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
bh_btpipein(data, verbose = True) # block_size_candidate should be >= 64 and 2의 승수.
# bh_btpipein(data, verbose = False) # block_size_candidate should be >= 64 and 2의 승수.
# fpga.WriteToBlockPipeIn(0x80, data[0:16384*2], reorder_str=True)
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################




fpga.CheckTriggered(0x60, 1, timeout=2.0) # config 다됐냐
# ######### P_STATE_05_DRAMFILL_INFERENCE_DATA mode ######################################################################





print('')
print('')
print('')
print('')












######### P_STATE_05_DRAMFILL_INFERENCE_DATA mode ######################################################################
이미 total inference dataset 파일이 존재하여 그것을 불러옵니다. path : zz_measure_dataset/nmnist/test/01_total_inference_data.txt
len(total_inference_dataset_hex_string): 7680000 x 4bit
dram_address_inference_last - dram_address_inference_start + 8: 960000 x 32bit

DRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!
dram_address_inference_start: 110,400
dram_address_inference_last: 1,070,392

write count before trigging (하위 17비트) address 재설정하면 0으로 리셋 돼 있음. trigging해보셈.
value = WireOut 0x20 value: 110400 unsigned hex: 0x1af40
write count after trigging (하위 17비트)
value = WireOut 0x20 value: 0 unsigned hex: 0x0


dram address 잘 드갔나?
dram_address_weight_start
value = WireOut 0x20 value: 110400 unsigned hex: 0x1af40
dram_address_weight_last
value = WireOut 0x20 value: 1070392 unsigned hex: 0x105538


dataset의 앞 64글자와 뒤 64글자
Start (MSB side): 00000000000000000000000000000000000000001c1000000000000000000

In [5]:
print('dram_address_weight_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_weight_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')


print('write count before trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
fpga.ActivateTriggerIn(0x40, 29)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
print('write count after trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('fifo_p2d_data_wr_cnt')
fpga.SetWireInValue(0x01, signed_encoding_32bit(12))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('쓴 데이터의 마지막 16bit')
fpga.SetWireInValue(0x01, signed_encoding_32bit(14))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# 수정 전: unsigned hex: 0xf659
# 수정 후: unsigned hex: f659 (딱 4자리)
string_from_fpga = f'{fpga.GetWireOutValue(0x20):04x}'
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {string_from_fpga}')
# print('넣은거  ', data[-4:])
# print(data)
print('넣은 거랑 똑같음?',str(data[-4:]) == str(string_from_fpga))
# print(f'len(data): {len(data)}')
if str(data[-4:]) != str(string_from_fpga):
    if  str(string_from_fpga) in str(data):
        #몇번째에
        idx = data.index(str(string_from_fpga))
        print(f'안에있음, {idx}번째에 있음')
        print(f'len(data) - idx: {len(data) - idx}')
    else:
        print('안에없음')
    

fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


dram_address_weight_start
value = WireOut 0x20 value: 110400 unsigned hex: 0x1af40
dram_address_weight_last
value = WireOut 0x20 value: 1070392 unsigned hex: 0x105538
write count before trigging (하위 17비트)
value = WireOut 0x20 value: 0 unsigned hex: 0x0
write count after trigging (하위 17비트)
value = WireOut 0x20 value: 42496 unsigned hex: 0xa600
fifo_p2d_data_wr_cnt
value = WireOut 0x20 value: 960000 unsigned hex: 0xea600
쓴 데이터의 마지막 16bit
value = WireOut 0x20 value: 0 unsigned hex: 0000
넣은 거랑 똑같음? True


0

## training data 삽입

In [6]:
print(f'######### P_STATE_06_DRAMFILL_TRAINING_DATA mode ######################################################################')
# ######### P_STATE_06_DRAMFILL_TRAINING_DATA mode ######################################################################
fpga.SetWireInValue(0x01, signed_encoding_32bit(6))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))



if dataset == 'DVS_GESTURE':
    saved_path = f"{training_data_path}/01_total_training_data_seed_{my_seed}.txt"
    if os.path.exists(saved_path) == True:
        print('이미 total training dataset 파일이 존재하여 그것을 불러옵니다.', f'path : {saved_path}')
        with open(saved_path, "r") as f:
            total_training_dataset_hex_string = f.read()
    else:
        print('total training dataset 파일이 존재하지 않아 새로 만듭니다.', f'path : {saved_path}')
        # 먼저 존재하는 파일들의 경로 리스트를 만듭니다.
        all_training_sample_paths = []
        for epoch in range(tr_epochs_in_dram):
            for sample_idx in range(tr_samples):
                sample_path = f"{training_data_path}/epoch_{epoch}/{file_name_prefix}_{sample_idx}.txt"
                if os.path.exists(sample_path):
                    all_training_sample_paths.append(sample_path)
                else:
                    assert False, f"파일을 찾을 수 없어: {sample_path}"

        # training making
        training_dataset = []
        for sample_path in all_training_sample_paths:
            with open(sample_path, "r") as f:
                sample_hex_string = f.read()
                training_dataset.append(sample_hex_string)
        total_training_dataset_hex_string = "".join(training_dataset)
        with open(saved_path, "w") as f:
            f.write(total_training_dataset_hex_string)

elif dataset == 'N_MNIST':
    print(f'NMNIST는 training data {tr_epochs_in_dram}epochs만')
    saved_path = f"{training_data_path}/01_total_training_data_seed_{my_seed}_epochs_{tr_epochs_in_dram}.txt"
    if os.path.exists(saved_path) == True:
        print('이미 total training dataset 파일이 존재하여 그것을 불러옵니다.', f'path : {saved_path}')
        with open(saved_path, "r") as f:
            total_training_dataset_hex_string = f.read()
    else:
        print('total training dataset 파일이 존재하지 않아 새로 만듭니다.', f'path : {saved_path}')
        # 먼저 존재하는 파일들의 경로 리스트를 만듭니다.
        all_training_sample_paths = []
        for sample_idx in range(tr_samples):
            sample_path = f"{training_data_path}/{file_name_prefix}_{sample_idx}.txt"
            if os.path.exists(sample_path):
                all_training_sample_paths.append(sample_path)
            else:
                assert False, f"파일을 찾을 수 없어: {sample_path}"

        # training making
        training_dataset = []
        for epoch in range(tr_epochs_in_dram):
            random.shuffle(all_training_sample_paths)
            for sample_path in all_training_sample_paths:
                with open(sample_path, "r") as f:
                    sample_hex_string = f.read()
                    training_dataset.append(sample_hex_string)
        total_training_dataset_hex_string = "".join(training_dataset)
        with open(saved_path, "w") as f:
            f.write(total_training_dataset_hex_string)

elif dataset == 'NTIDIGITS':
    saved_path = f"{training_data_path}/01_total_training_data_seed_{my_seed}.txt"
    if os.path.exists(saved_path) == True:
        print('이미 total training dataset 파일이 존재하여 그것을 불러옵니다.', f'path : {saved_path}')
        with open(saved_path, "r") as f:
            total_training_dataset_hex_string = f.read()
    else:
        print('total training dataset 파일이 존재하지 않아 새로 만듭니다.', f'path : {saved_path}')
        # 먼저 존재하는 파일들의 경로 리스트를 만듭니다.
        all_training_sample_paths = []
        for sample_idx in range(tr_samples):
            sample_path = f"{training_data_path}/{file_name_prefix}_{sample_idx}.txt"
            if os.path.exists(sample_path):
                all_training_sample_paths.append(sample_path)
            else:
                assert False, f"파일을 찾을 수 없어: {sample_path}"

        # training making
        training_dataset = []
        for epoch in range(tr_epochs_in_dram):
            random.shuffle(all_training_sample_paths)
            for sample_path in all_training_sample_paths:
                with open(sample_path, "r") as f:
                    sample_hex_string = f.read()
                    training_dataset.append(sample_hex_string)
        total_training_dataset_hex_string = "".join(training_dataset)
        with open(saved_path, "w") as f:
            f.write(total_training_dataset_hex_string)













# # for test
# # for test
# # for test
# # for test
# total_training_dataset_hex_string = token_hex(512//8) # byte
# dram_address_training_start = dram_address_training_start
# dram_address_training_last = dram_address_training_start + (512//32) - 8
# # for test
# # for test
# # for test















with open("total_training_dataset_hex_string.txt", "w") as f:
    f.write(total_training_dataset_hex_string)

with open("total_training_dataset_hex_string.txt", "r") as f:
    total_training_dataset_hex_string = f.read()

print(f'len(total_training_dataset_hex_string): {len(total_training_dataset_hex_string)} x 4bit')
print(f'dram_address_training_last - dram_address_training_start + 8: {dram_address_training_last - dram_address_training_start + 8} x 32bit')






print(f'\nDRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!')
print(f'dram_address_training_start: {dram_address_training_start:,}')
print(f'dram_address_training_last: {dram_address_training_last:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_training_start)) # dram write address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_training_last)) # dram write address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐


print('\nwrite count before trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
fpga.ActivateTriggerIn(0x40, 29)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
print('write count after trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')




print('\n\ndram address 잘 드갔나?')
print('dram_address_weight_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_weight_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
print('\n')


fpga.SetWireInValue(0x01, signed_encoding_32bit(0))








print('dataset의 앞 64글자와 뒤 64글자')
print(f"Start (MSB side): {total_training_dataset_hex_string[:64]}...")
print(f"End (LSB side): ...{total_training_dataset_hex_string[-64:]}")

data = total_training_dataset_hex_string







############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
# bh_btpipein(data, verbose = True) # block_size_candidate should be >= 64 and 2의 승수.
# bh_btpipein(data, verbose = False) # block_size_candidate should be >= 64 and 2의 승수.

if len(data) % 16*8 == 0:
    for slice_idx in range(8):
        bh_btpipein(data[(len(data)//8)*slice_idx:(len(data)//8)*(slice_idx+1)], verbose = True) # block_size_candidate should be >= 64 and 2의 승수.
else:
    assert False, f"데이터 길이가 16*8의 배수가 아님, len(data): {len(data)}"
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################
############## PIPE IN !!!!!!!!!!!!!!!!!! #####################




fpga.CheckTriggered(0x60, 1, timeout=60.0) # config 다됐냐
# ######### P_STATE_06_DRAMFILL_TRAINING_DATA mode ######################################################################





print('\n\n DRAM FILL FINISH')


print('')
print('')
print('')
print('')















######### P_STATE_06_DRAMFILL_TRAINING_DATA mode ######################################################################
NMNIST는 training data 25epochs만
이미 total training dataset 파일이 존재하여 그것을 불러옵니다. path : zz_measure_dataset/nmnist/train/01_total_training_data_seed_0_epochs_25.txt
len(total_training_dataset_hex_string): 1152000000 x 4bit
dram_address_training_last - dram_address_training_start + 8: 144000000 x 32bit

DRAM WRITE ADDRESS CONFIG!!!!!!!!!!!!
dram_address_training_start: 1,070,400
dram_address_training_last: 145,070,392

write count before trigging (하위 17비트)
value = WireOut 0x20 value: 42496 unsigned hex: 0xa600
write count after trigging (하위 17비트)
value = WireOut 0x20 value: 0 unsigned hex: 0x0


dram address 잘 드갔나?
dram_address_weight_start
value = WireOut 0x20 value: 1070400 unsigned hex: 0x105540
dram_address_weight_last
value = WireOut 0x20 value: 145070392 unsigned hex: 0x8a59938


dataset의 앞 64글자와 뒤 64글자
Start (MSB side): 0000000000000000000000000000000000000000240080

In [7]:
print('dram_address_weight_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_weight_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')


print('write count before trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
fpga.ActivateTriggerIn(0x40, 29)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
print('write count after trigging', '(하위 17비트)')
fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('fifo_p2d_data_wr_cnt')
fpga.SetWireInValue(0x01, signed_encoding_32bit(12))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('쓴 데이터의 마지막 16bit')
fpga.SetWireInValue(0x01, signed_encoding_32bit(14))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# 수정 전: unsigned hex: 0xf659
# 수정 후: unsigned hex: f659 (딱 4자리)
string_from_fpga = f'{fpga.GetWireOutValue(0x20):04x}'
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {string_from_fpga}')
# print('넣은거  ', data[-4:])
# print(data)
print('넣은 거랑 똑같음?',str(data[-4:]) == str(string_from_fpga))
# print(f'len(data): {len(data)}')
if str(data[-4:]) != str(string_from_fpga):
    if  str(string_from_fpga) in str(data):
        #몇번째에
        idx = data.index(str(string_from_fpga))
        print(f'안에있음, {idx}번째에 있음')
        print(f'len(data) - idx: {len(data) - idx}')
    else:
        print('안에없음')
    

fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


dram_address_weight_start
value = WireOut 0x20 value: 1070400 unsigned hex: 0x105540
dram_address_weight_last
value = WireOut 0x20 value: 145070392 unsigned hex: 0x8a59938
write count before trigging (하위 17비트)
value = WireOut 0x20 value: 0 unsigned hex: 0x0
write count after trigging (하위 17비트)
value = WireOut 0x20 value: 82944 unsigned hex: 0x14400
fifo_p2d_data_wr_cnt
value = WireOut 0x20 value: 144000000 unsigned hex: 0x8954400
쓴 데이터의 마지막 16bit
value = WireOut 0x20 value: 0 unsigned hex: 0000
넣은 거랑 똑같음? True


0

## ASIC에 CONFIG 밀어 넣기

In [8]:
print(f'######### P_STATE_07_ASIC_CONFIG mode ######################################################################')
# ######### P_STATE_07_ASIC_CONFIG mode ######################################################################
fpga.SetWireInValue(0x01, signed_encoding_32bit(7))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))




print(f'\nDRAM WEIGHT READ ADDRESS CONFIG!!!!!!!!!!!!')
print(f'dram_address_weight_start: {dram_address_weight_start:,}')
print(f'dram_address_weight_last: {dram_address_weight_last:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_weight_start)) # dram write address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_weight_last)) # dram write address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐



print('\n\ndram address 잘 드갔나?')
print('dram_address_weight_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_weight_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
print('\n')



streaming_wait_cycle = 4
assert 0 <= streaming_wait_cycle
print(f'STREAMING WAIT CYCLE 설정: 7번 넣고 {streaming_wait_cycle}번 사이클 기다림.')
fpga.SetWireInValue(0x01, signed_encoding_32bit(streaming_wait_cycle))
fpga.ActivateTriggerIn(0x40, 27)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))




# ASIC start ready 확인
fpga.ActivateTriggerIn(0x40, 28)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'\nASIC start ready 확인 = WireOut 0x20 value: {ep20wireout} (1이면 준비된 것) unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
assert ep20wireout == 1, "ASIC start 준비가 안된 상태임"






# config 시작!!!
print(f'\nASIC CONFIG 시작!')
fpga.ActivateTriggerIn(0x40, 0)
fpga.CheckTriggered(0x60, 1, timeout=5.0) # config 다됐냐
print(f'\nASIC CONFIG 종료!')




fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'a_domain에서 66bit stream 몇번? {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))

# ######### P_STATE_07_ASIC_CONFIG mode ######################################################################








print('')
print('')
print('')
print('')
















######### P_STATE_07_ASIC_CONFIG mode ######################################################################

DRAM WEIGHT READ ADDRESS CONFIG!!!!!!!!!!!!
dram_address_weight_start: 0
dram_address_weight_last: 110,392


dram address 잘 드갔나?
dram_address_weight_start
value = WireOut 0x20 value: 0 unsigned hex: 0x0
dram_address_weight_last
value = WireOut 0x20 value: 110392 unsigned hex: 0x1af38


STREAMING WAIT CYCLE 설정: 7번 넣고 4번 사이클 기다림.

ASIC start ready 확인 = WireOut 0x20 value: 1 (1이면 준비된 것) unsigned hex: 0x1

ASIC CONFIG 시작!

ASIC CONFIG 종료!
a_domain에서 66bit stream 몇번? 41424 unsigned hex: 0xa1d0






## TRANING

In [9]:
print(f'######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################')
# ######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################

print(f'\nDRAM READ ADDRESS CONFIG!!!!!!!!!!!!')
print(f'dram_address_training_start: {dram_address_training_start:,}')
print(f'dram_address_training_last: {dram_address_training_last:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_training_start)) # dram READ address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_training_last)) # dram READ address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐



print('\n\ndram address 잘 드갔나?')
print('dram_address_read_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_read_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
print('\n')
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


EPOCH = 200
THIS_TRY_SAMPLES = EPOCH * tr_samples
print(f'\nsample 수 삽입 (training epochs x training samples per epoch) {EPOCH} X {tr_samples:,} = {THIS_TRY_SAMPLES:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(THIS_TRY_SAMPLES)) 
fpga.ActivateTriggerIn(0x40, 26)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=5.0) 

fpga.SetWireInValue(0x01, signed_encoding_32bit(2))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
print(f'sample수 삽입 확인 = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')


fpga.SetWireInValue(0x01, signed_encoding_32bit(18))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
print(f'wirein18로 디버깅 {bin(ep20wireout)} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

# ######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################








print('')
print('')
print('')
print('')














print(f'######### P_STATE_11_ASIC_TRAINING_QUEUING mode ######################################################################')
# ######### P_STATE_11_ASIC_TRAINING_QUEUING mode ######################################################################

print(f'\nQUEUING 시작')
fpga.ActivateTriggerIn(0x40, 0) # 0 training, 1 inference
fpga.CheckTriggered(0x60, 1, timeout=5.0) # queuing 다 됐나
print(f'QUEUING 완료\n')


# ASIC start ready 확인
fpga.ActivateTriggerIn(0x40, 28)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'ASIC start ready 확인 = WireOut 0x20 value: {ep20wireout} (1이면 준비된 것) unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
assert ep20wireout == 1, "ASIC start 준비가 안된 상태임"
# ######### P_STATE_11_ASIC_TRAINING_QUEUING mode ######################################################################













print('')
print('')
print('')
print('')



print(f'######### P_STATE_12_ASIC_TRAINING_PROCESSING mode ######################################################################')
# ######### P_STATE_12_ASIC_TRAINING_PROCESSING mode ######################################################################

print(f'\nPROCESSING 시작')
time_start = time.time()
fpga.ActivateTriggerIn(0x40, 0) # GO PROCESSING
fpga.CheckTriggered(0x60, 1, timeout=1000.0) # processing완료
time_end = time.time()
print(f'\nPROCESSING 종료, 걸린 시간: {time_end - time_start:.2f} seconds\n')



# sample_num_executed_from_a
fpga.ActivateTriggerIn(0x40, 22)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(19))
samples_executed_from_a = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
assert samples_executed_from_a == THIS_TRY_SAMPLES, f"처리한 sample 수가 삽입한 수랑 다름! ep20wireout: {ep20wireout:,} vs THIS_TRY_SAMPLES: {THIS_TRY_SAMPLES:,}"
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
print(f'sample_num_executed_from_a: {ep20wireout:,}')

# ######### P_STATE_12_ASIC_TRAINING_PROCESSING mode ######################################################################


# @ P_STATE_08_ASIC_CONFIG_DONE
print(f'\PROCESSING TIME 세팅 하기')
fpga.ActivateTriggerIn(0x40, 23) # GO PROCESSING
fpga.CheckTriggered(0x60, 1, timeout=1.0) # processing완료

fpga.SetWireInValue(0x01, signed_encoding_32bit(16))
PROCESSING_CLK_CNT_LOW32BIT = fpga.GetWireOutValue(0x20) # signed 아니니까 signed decoding하지마
fpga.SetWireInValue(0x01, signed_encoding_32bit(17))
PROCESSING_CLK_CNT_HIGH32BIT = fpga.GetWireOutValue(0x20) # signed 아니니까 signed decoding하지마
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
TOTAL_PROCESSING_CLK_CNT = (PROCESSING_CLK_CNT_HIGH32BIT * (2**32)) + PROCESSING_CLK_CNT_LOW32BIT
print(f'총 PROCESSING CYCLE 수: {TOTAL_PROCESSING_CLK_CNT:,} cycles')
print(f'PROCESSING CYCLE / SAMPLE: {TOTAL_PROCESSING_CLK_CNT/samples_executed_from_a:.2} cycles')
ASIC_FREQUENCY_MHZ = 20
print(f'ASIC {ASIC_FREQUENCY_MHZ}MHz라면 샘플 당 걸리는 시간: {(TOTAL_PROCESSING_CLK_CNT/samples_executed_from_a) * (1000/ASIC_FREQUENCY_MHZ) * 0.001:.2f} us')



# print(f'd domain 상태보기')
# fpga.ActivateTriggerIn(0x40, 20)
# fpga.CheckTriggered(0x60, 1, timeout=1.0)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(20))
# label0_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(21))
# label1_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(22))
# label2_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(23))
# label3_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(24))
# label4_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(25))
# label5_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(26))
# label6_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(27))
# label7_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(28))
# label8_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(29))
# label9_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(30))
# epoch_finish_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(31))
# sample_finish_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))

# print(f'd domain 결과:')
# print(f'label0_num: {label0_num:,}')
# print(f'label1_num: {label1_num:,}')
# print(f'label2_num: {label2_num:,}')
# print(f'label3_num: {label3_num:,}')
# print(f'label4_num: {label4_num:,}')
# print(f'label5_num: {label5_num:,}')
# print(f'label6_num: {label6_num:,}')
# print(f'label7_num: {label7_num:,}')
# print(f'label8_num: {label8_num:,}')
# print(f'label9_num: {label9_num:,}')
# print(f'epoch_finish_num: {epoch_finish_num:,}')
# print(f'sample_finish_num: {sample_finish_num:,}')

# ######### P_STATE_10_ASIC_INFERENCE_PROCESSING mode ######################################################################







# ######### P_STATE_12_ASIC_TRAINING_PROCESSING mode ######################################################################







######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################

DRAM READ ADDRESS CONFIG!!!!!!!!!!!!
dram_address_training_start: 1,070,400
dram_address_training_last: 145,070,392


dram address 잘 드갔나?
dram_address_read_start
value = WireOut 0x20 value: 1070400 unsigned hex: 0x105540
dram_address_read_last
value = WireOut 0x20 value: 145070392 unsigned hex: 0x8a59938



sample 수 삽입 (training epochs x training samples per epoch) 200 X 60,000 = 12,000,000
sample수 삽입 확인 = WireOut 0x20 value: 12000000 unsigned hex: 0x1
wirein18로 디버깅 0b101100000 unsigned hex: 0x1




######### P_STATE_11_ASIC_TRAINING_QUEUING mode ######################################################################

QUEUING 시작
QUEUING 완료

ASIC start ready 확인 = WireOut 0x20 value: 1 (1이면 준비된 것) unsigned hex: 0x1




######### P_STATE_12_ASIC_TRAINING_PROCESSING mode ######################################################################

PROCESSING 시작

PROCESSING 종료, 걸린 시간:

In [10]:

# # sample_num_executed_from_a
# fpga.ActivateTriggerIn(0x40, 22)
# fpga.CheckTriggered(0x60, 1, timeout=1.0)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# print(f'{ep20wireout:,}')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(19))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# print(f'{ep20wireout:,}')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(21))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# print(f'{ep20wireout:,}')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(22))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# print(f'{ep20wireout:,}')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(23))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# print(f'{ep20wireout:,}')

## INFERENCE

In [11]:
print(f'######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################')
# ######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################

print(f'\nDRAM READ ADDRESS CONFIG!!!!!!!!!!!!')
print(f'dram_address_inference_start: {dram_address_inference_start:,}')
print(f'dram_address_inference_last: {dram_address_inference_last:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_inference_start)) # dram READ address
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(dram_address_inference_last)) # dram READ address last
fpga.ActivateTriggerIn(0x40, 30)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐



print('\n\ndram address 잘 드갔나?')
print('dram_address_read_start')
fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

print('dram_address_read_last')
fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
print('\n')
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))



EPOCH = 200
THIS_TRY_SAMPLES = EPOCH * inf_samples
print(f'\nsample 수 삽입 (inference epochs x inference samples per epoch) {EPOCH} X {inf_samples:,} = {THIS_TRY_SAMPLES:,}')
fpga.SetWireInValue(0x01, signed_encoding_32bit(THIS_TRY_SAMPLES))
fpga.ActivateTriggerIn(0x40, 26)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
fpga.CheckTriggered(0x60, 1, timeout=1.0) # config 다됐냐

# ######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################








print('')
print('')
print('')
print('')














print(f'######### P_STATE_09_ASIC_INFERENCE_QUEUING mode ######################################################################')
# ######### P_STATE_09_ASIC_INFERENCE_QUEUING mode ######################################################################

print(f'\nQUEUING 시작')
fpga.ActivateTriggerIn(0x40, 1) # 0 training, 1 inference
fpga.CheckTriggered(0x60, 1, timeout=5.0) # queuing 다 됐나
print(f'QUEUING 완료\n')


# ASIC start ready 확인
fpga.ActivateTriggerIn(0x40, 28)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'ASIC start ready 확인 = WireOut 0x20 value: {ep20wireout} (1이면 준비된 것) unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
assert ep20wireout == 1, "ASIC start 준비가 안된 상태임"
# ######### P_STATE_09_ASIC_INFERENCE_QUEUING mode ######################################################################


print('')
print('')
print('')
print('')



print(f'######### P_STATE_10_ASIC_INFERENCE_PROCESSING mode ######################################################################')
# ######### P_STATE_10_ASIC_INFERENCE_PROCESSING mode ######################################################################

print(f'\nPROCESSING 시작')
time_start = time.time()
fpga.ActivateTriggerIn(0x40, 0) # GO PROCESSING
fpga.CheckTriggered(0x60, 1, timeout=60.0) # processing완료
time_end = time.time()
print(f'\nPROCESSING 종료, 걸린 시간: {time_end - time_start:.2f} seconds\n')
# sample_num_executed_from_a
fpga.ActivateTriggerIn(0x40, 22)
fpga.CheckTriggered(0x60, 1, timeout=1.0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(19))
samples_executed_from_a = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
assert samples_executed_from_a == THIS_TRY_SAMPLES, f"처리한 sample 수가 삽입한 수랑 다름! ep20wireout: {ep20wireout:,} vs THIS_TRY_SAMPLES: {THIS_TRY_SAMPLES:,}"
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
print(f'sample_num_executed_from_a: {ep20wireout:,}')

# @ P_STATE_08_ASIC_CONFIG_DONE
print(f'\n결과 세팅 하기')
fpga.ActivateTriggerIn(0x40, 25) # GO PROCESSING
fpga.CheckTriggered(0x60, 1, timeout=1.0) # processing완료

fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
correct_sample_num = ep20wireout
fpga.SetWireInValue(0x01, signed_encoding_32bit(14))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
wrong_sample_num = ep20wireout
fpga.SetWireInValue(0x01, signed_encoding_32bit(15))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
total_inference_sample_num = ep20wireout
assert total_inference_sample_num == THIS_TRY_SAMPLES, f"처리한 sample 수가 삽입한 수랑 다름! ep20wireout: {ep20wireout:,} vs THIS_TRY_SAMPLES: {THIS_TRY_SAMPLES:,}"

fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
print(f'결과: correct_sample_num: {correct_sample_num}, wrong_sample_num: {wrong_sample_num}, total_inference_sample_num: {total_inference_sample_num}')
print(f'정확도: {(correct_sample_num/total_inference_sample_num)*100:.2f} %')
assert correct_sample_num + wrong_sample_num == total_inference_sample_num, "맞은거 + 틀린거 != 전체 샘플 수"



# @ P_STATE_08_ASIC_CONFIG_DONE
print(f'\PROCESSING TIME 세팅 하기')
fpga.ActivateTriggerIn(0x40, 23)
fpga.CheckTriggered(0x60, 1, timeout=1.0)

fpga.SetWireInValue(0x01, signed_encoding_32bit(16))
PROCESSING_CLK_CNT_LOW32BIT = fpga.GetWireOutValue(0x20) # signed 아니니까 signed decoding하지마
fpga.SetWireInValue(0x01, signed_encoding_32bit(17))
PROCESSING_CLK_CNT_HIGH32BIT = fpga.GetWireOutValue(0x20) # signed 아니니까 signed decoding하지마
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
TOTAL_PROCESSING_CLK_CNT = (PROCESSING_CLK_CNT_HIGH32BIT * (2**32)) + PROCESSING_CLK_CNT_LOW32BIT
print(f'총 PROCESSING CYCLE 수: {TOTAL_PROCESSING_CLK_CNT:,} cycles')
print(f'PROCESSING CYCLE / SAMPLE: {TOTAL_PROCESSING_CLK_CNT/total_inference_sample_num:.2} cycles')
ASIC_FREQUENCY_MHZ = 20
print(f'ASIC {ASIC_FREQUENCY_MHZ}MHz라면 샘플 당 걸리는 시간: {(TOTAL_PROCESSING_CLK_CNT/total_inference_sample_num) * (1000/ASIC_FREQUENCY_MHZ) * 0.001:.2f} us')



# print(f'd domain 상태보기')
# fpga.ActivateTriggerIn(0x40, 20)
# fpga.CheckTriggered(0x60, 1, timeout=1.0)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(20))
# label0_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(21))
# label1_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(22))
# label2_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(23))
# label3_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(24))
# label4_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(25))
# label5_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(26))
# label6_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(27))
# label7_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(28))
# label8_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(29))
# label9_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(30))
# epoch_finish_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(31))
# sample_finish_num = fpga.GetWireOutValue(0x20)
# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))

# print(f'd domain 결과:')
# print(f'label0_num: {label0_num:,}')
# print(f'label1_num: {label1_num:,}')
# print(f'label2_num: {label2_num:,}')
# print(f'label3_num: {label3_num:,}')
# print(f'label4_num: {label4_num:,}')
# print(f'label5_num: {label5_num:,}')
# print(f'label6_num: {label6_num:,}')
# print(f'label7_num: {label7_num:,}')
# print(f'label8_num: {label8_num:,}')
# print(f'label9_num: {label9_num:,}')
# print(f'epoch_finish_num: {epoch_finish_num:,}')
# print(f'sample_finish_num: {sample_finish_num:,}')

# ######### P_STATE_10_ASIC_INFERENCE_PROCESSING mode ######################################################################







######### P_STATE_08_ASIC_CONFIG_DONE mode ######################################################################

DRAM READ ADDRESS CONFIG!!!!!!!!!!!!
dram_address_inference_start: 110,400
dram_address_inference_last: 1,070,392


dram address 잘 드갔나?
dram_address_read_start
value = WireOut 0x20 value: 110400 unsigned hex: 0x1af40
dram_address_read_last
value = WireOut 0x20 value: 1070392 unsigned hex: 0x105538



sample 수 삽입 (inference epochs x inference samples per epoch) 200 X 10,000 = 2,000,000




######### P_STATE_09_ASIC_INFERENCE_QUEUING mode ######################################################################

QUEUING 시작
QUEUING 완료

ASIC start ready 확인 = WireOut 0x20 value: 1 (1이면 준비된 것) unsigned hex: 0x1




######### P_STATE_10_ASIC_INFERENCE_PROCESSING mode ######################################################################

PROCESSING 시작

PROCESSING 종료, 걸린 시간: 40.31 seconds

sample_num_executed_from_a: 1

결과 세팅 하기
결과: correct_sample_num: 1922600, wrong_sample_num: 7740

In [12]:
# import os
# from collections import Counter

# # 설정
# base_path = "zz_measure_dataset/nmnist/test/"
# label_counts = Counter()
# total_processed = 0

# print("데이터 분석 중...")

# for i in range(10000):
#     file_name = f"nmnist_{i}.txt"
#     file_path = os.path.join(base_path, file_name)
    
#     try:
#         if os.path.exists(file_path):
#             with open(file_path, 'r') as f:
#                 hex_data = f.read().strip()
            
#             # 비트 162:166을 추출하려면 최소 42글자의 16진수가 필요합니다 (42 * 4 = 168비트)
#             # 성능을 위해 앞의 50글자만 가져와서 변환합니다.
#             hex_subset = hex_data[:50]
#             binary_data = "".join(bin(int(c, 16))[2:].zfill(4) for c in hex_subset)
            
#             if len(binary_data) >= 166:
#                 # 162번부터 4비트 추출 (162, 163, 164, 165)
#                 label_bits = binary_data[162:166]
#                 label_int = int(label_bits, 2) # 이진수 문자열을 정수로 변환
                
#                 label_counts[label_int] += 1
#                 total_processed += 1
#         else:
#             # 파일이 중간에 비어있을 경우를 대비해 pass
#             continue
            
#     except Exception as e:
#         print(f"파일 {file_name} 처리 중 오류 발생: {e}")

# # 결과 출력
# print("\n" + "="*30)
# print(f"{'라벨(Int)':<10} | {'빈도수':<10}")
# print("-" * 30)
# for label in sorted(label_counts.keys()):
#     print(f"{label:<10} | {label_counts[label]:<10}")
# print("-" * 30)
# print(f"{'총 처리 파일':<10} | {total_processed:<10}")
# print("="*30)




# # 데이터 분석 중...

# # ==============================
# # 라벨(Int)    | 빈도수       
# # ------------------------------
# # 0          | 980       
# # 1          | 1135      
# # 2          | 1032      
# # 3          | 1010      
# # 4          | 982       
# # 5          | 892       
# # 6          | 958       
# # 7          | 1028      
# # 8          | 974       
# # 9          | 1009      
# # ------------------------------
# # 총 처리 파일    | 10000     
# # ==============================






# import os
# from collections import Counter

# # 설정
# base_path = "zz_measure_dataset/gesture/topN_4/seed_0/test/"
# label_counts = Counter()
# total_processed = 0

# print("Gesture 데이터 분석 중...")

# for i in range(240): # 0부터 239까지
#     file_name = f"gesture_{i}.txt"
#     file_path = os.path.join(base_path, file_name)
    
#     try:
#         if os.path.exists(file_path):
#             with open(file_path, 'r') as f:
#                 hex_data = f.read().strip()
            
#             # 비트 1128~1131을 추출하기 위해 넉넉하게 앞의 300글자 이상을 가져옵니다.
#             # 16진수 300자 = 1200비트
#             hex_subset = hex_data[:300]
#             binary_data = "".join(bin(int(c, 16))[2:].zfill(4) for c in hex_subset)
            
#             if len(binary_data) >= 1132:
#                 # 라벨 추출: 1128부터 4비트 (1128, 1129, 1130, 1131)
#                 label_bits = binary_data[1128:1132]
#                 label_int = int(label_bits, 2)
                
#                 label_counts[label_int] += 1
#                 total_processed += 1
#         else:
#             continue
            
#     except Exception as e:
#         print(f"파일 {file_name} 처리 중 오류 발생: {e}")

# # 결과 출력
# print("\n" + "="*35)
# print(f"{'Gesture 라벨':<12} | {'빈도수':<10}")
# print("-" * 35)
# for label in sorted(label_counts.keys()):
#     print(f"Label {label:<6} | {label_counts[label]:<10}")
# print("-" * 35)
# print(f"{'총 처리 파일':<12} | {total_processed:<10}")
# print("="*35)

# Gesture 데이터 분석 중...

# ===================================
# Gesture 라벨   | 빈도수       
# -----------------------------------
# Label 0      | 24        
# Label 1      | 24        
# Label 2      | 24        
# Label 3      | 24        
# Label 4      | 24        
# Label 5      | 24        
# Label 6      | 24        
# Label 7      | 24        
# Label 8      | 24        
# Label 9      | 24        
# -----------------------------------
# 총 처리 파일      | 240       
# ===================================


















# import os
# from collections import Counter

# # 설정
# base_path = "zz_measure_dataset/ntidigits/word0/test/"
# label_counts = Counter()
# total_processed = 0

# print("NTIDIGITS 데이터 분석 중 (라벨 비트: 224-227)...")

# # 0부터 451까지 샘플 순회
# for i in range(452):
#     file_name = f"ntidigits_word0_{i}.txt"
#     file_path = os.path.join(base_path, file_name)
    
#     try:
#         if os.path.exists(file_path):
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 hex_data = f.read().strip()
            
#             # 비트 224~227을 포함하기 위해 앞부분 16진수 70자 추출 (280비트 분량)
#             hex_subset = hex_data[:70]
#             binary_data = "".join(bin(int(c, 16))[2:].zfill(4) for c in hex_subset)
            
#             if len(binary_data) >= 228:
#                 # 라벨 추출: 224번부터 227번 비트까지 (슬라이싱 [224:228])
#                 label_bits = binary_data[224:228]
#                 label_int = int(label_bits, 2)
                
#                 label_counts[label_int] += 1
#                 total_processed += 1
#         else:
#             continue
            
#     except Exception as e:
#         print(f"파일 {file_name} 처리 중 오류 발생: {e}")

# # 결과 출력
# print("\n" + "="*35)
# print(f"{'NTIDIGITS 라벨':<14} | {'빈도수':<10}")
# print("-" * 35)
# for label in sorted(label_counts.keys()):
#     print(f"Label {label:<8} | {label_counts[label]:<10}")
# print("-" * 35)
# print(f"{'총 처리 파일':<14} | {total_processed:<10}")
# print("="*35)

# import os
# from collections import Counter

# # 설정
# base_path = "zz_measure_dataset/ntidigits/word0/test/"
# label_counts = Counter()
# total_processed = 0

# print("NTIDIGITS 데이터 분석 중 (라벨 비트: 224-227)...")

# # 0부터 451까지 샘플 순회
# for i in range(452):
#     file_name = f"ntidigits_word0_{i}.txt"
#     file_path = os.path.join(base_path, file_name)
    
#     try:
#         if os.path.exists(file_path):
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 hex_data = f.read().strip()
            
#             # 비트 224~227을 포함하기 위해 앞부분 16진수 70자 추출 (280비트 분량)
#             hex_subset = hex_data[:70]
#             binary_data = "".join(bin(int(c, 16))[2:].zfill(4) for c in hex_subset)
            
#             if len(binary_data) >= 228:
#                 # 라벨 추출: 224번부터 227번 비트까지 (슬라이싱 [224:228])
#                 label_bits = binary_data[224:228]
#                 label_int = int(label_bits, 2)
                
#                 label_counts[label_int] += 1
#                 total_processed += 1
#         else:
#             continue
            
#     except Exception as e:
#         print(f"파일 {file_name} 처리 중 오류 발생: {e}")

# # 결과 출력
# print("\n" + "="*35)
# print(f"{'NTIDIGITS 라벨':<14} | {'빈도수':<10}")
# print("-" * 35)
# for label in sorted(label_counts.keys()):
#     print(f"Label {label:<8} | {label_counts[label]:<10}")
# print("-" * 35)
# print(f"{'총 처리 파일':<14} | {total_processed:<10}")
# print("="*35)

In [13]:
# # fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# # fpga.SetWireInValue(0x01, signed_encoding_32bit(1))


# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')

In [14]:
# fpga.close() 

In [15]:
# print('dram_address_weight_start')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(10))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

# print('dram_address_weight_last')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(11))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')


# print('write count before trigging', '(하위 17비트)')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')
# fpga.ActivateTriggerIn(0x40, 29)
# fpga.CheckTriggered(0x60, 1, timeout=1.0)
# print('write count after trigging', '(하위 17비트)')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(13))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

# print('fifo_p2d_data_wr_cnt')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(12))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {hex(fpga.GetWireOutValue(0x20))}')

# print('쓴 데이터의 마지막 16bit')
# fpga.SetWireInValue(0x01, signed_encoding_32bit(14))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# # 수정 전: unsigned hex: 0xf659
# # 수정 후: unsigned hex: f659 (딱 4자리)
# string_from_fpga = f'{fpga.GetWireOutValue(0x20):04x}'
# print(f'value = WireOut 0x20 value: {ep20wireout} unsigned hex: {string_from_fpga}')
# # print('넣은거  ', data[-4:])
# # print(data)
# print('넣은 거랑 똑같음?',str(data[-4:]) == str(string_from_fpga))
# # print(f'len(data): {len(data)}')
# if str(data[-4:]) != str(string_from_fpga):
#     if  str(string_from_fpga) in str(data):
#         #몇번째에
#         idx = data.index(str(string_from_fpga))
#         print(f'안에있음, {idx}번째에 있음')
#         print(f'len(data) - idx: {len(data) - idx}')
#     else:
#         print('안에없음')
    

# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
